In [ ]:
import holoviews as hv
import numpy as np

from holoviews import opts
from holoviews.streams import RangeXY
from bokeh.models import HoverTool
from pathlib import PurePath

from raster_compare.base import PixelClassifier, RasterFile

hv.extension('bokeh', 'matplotlib')

HOME_DIR = PurePath('/Volumes/warehouse/projects/UofU/ASO/ERW')

In [ ]:
casi_data = PixelClassifier(
    HOME_DIR / 'ERW_20180524/CASI/CASI_2018_05_24_weekly_mosaic_classified_lidargrid_4m.tif',
    HOME_DIR / 'cloud_mask/cloud_mask_lidargrid_4m.tif',
)

return_diff = RasterFile(
    HOME_DIR / 'ERW_20180524/Lidar/ERW_20180524_Lidar_fr__lr_diff_4m.tif', 
    band_number=1
)

return_diff_values = return_diff.band_values()
return_diff_values_mask = return_diff_values.mask.copy()
return_diff_values.mask = casi_data.snow_surfaces(return_diff_values_mask)

In [ ]:
hv_raster = hv.Dataset(
    (range(return_diff_values.shape[1]), range(return_diff_values.shape[0]), 
    np.flip(return_diff_values, axis=0)),
    ['x', 'y'], 'difference'
)

In [ ]:
plot = hv_raster.to(hv.Image, ['x', 'y']).redim.range(difference=(-3, 3))

In [ ]:
hover = HoverTool(
    tooltips=[
        ("Difference", "@image{%0.3f}")
    ],
    formatters={
        'image': 'printf'
    }
)
clip_colors = { 'NaN': 'lightgray', 'min': 'red', 'max': 'blue'}

plot.opts(
    opts.Image(
        height=840, width=700, 
        cmap='RdBu', symmetric=True, clipping_colors=clip_colors,
        tools=[hover], 
        colorbar=True
    )
)

In [ ]:
def selected_hist(x_range, y_range):
    obj = plot.select(x=x_range, y=y_range) if x_range and y_range else plot
    return hv.operation.histogram(obj).opts(width=700)

diff_range_xy = hv.streams.RangeXY(source=plot)
layout = hv.Layout(plot + hv.DynamicMap(selected_hist, streams=[diff_range_xy])).cols(2)

In [ ]:
layout